# Inspect your runnables

In [ ]:
import os

API_KEY = os.getenv("UNION_API_KEY")
BASE_URL = os.getenv("UNION_BASE_URL")

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [2]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## 获取`Runnable`图

In [4]:
chain.get_graph()

Graph(nodes={'9157464ad7994c2282445cdb316136f2': Node(id='9157464ad7994c2282445cdb316136f2', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Input'>), '8d17865d2424480aa6dcace31c696dd4': Node(id='8d17865d2424480aa6dcace31c696dd4', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Output'>), 'b77b410d95224979be24421d39e1c2ba': Node(id='b77b410d95224979be24421d39e1c2ba', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000191E747A030>)), '69d04f0d5b98472396279280b60e1333': Node(id='69d04f0d5b98472396279280b60e1333', data=RunnablePassthrough()), 'adecb0cca8ba47359c685bc2ef4cac43': Node(id='adecb0cca8ba47359c685bc2ef4cac43', data=ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: 

## 打印`Runnable`图

In [7]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

## 获取`Prompt`

In [8]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])]